In [65]:
import numpy as np
import random

from my_q1_softmax import softmax
from my_q2_sigmoid import sigmoid, sigmoid_grad
from my_q2_gradcheck import gradcheck_naive

In [66]:
from scipy.stats import logistic
def forward_backprop(data, labels, params, dimensions):
    ofs = 0
    Dx, H, Dy = (dimensions[0], dimensions[1], dimensions[2])

    W1 = np.reshape(params[ofs:ofs+ Dx * H], (Dx, H))
    ofs += Dx * H
    b1 = np.reshape(params[ofs:ofs + H], (1, H))
    ofs += H
    W2 = np.reshape(params[ofs:ofs + H * Dy], (H, Dy))
    ofs += H * Dy
    b2 = np.reshape(params[ofs:ofs + Dy], (1, Dy))
    h=sigmoid(np.dot(data,W1)+b1)
    y_pred=softmax(np.dot(h,W2)+b2)

    # Back-Propagation
    gradtheta=y_pred-labels
    gradW2=np.dot(h.T,gradtheta)
    gradb2=y_pred-labels
    gradh=np.dot(gradtheta,W2.T)
    #gradsigmoid=np.multiply(gradh,sigmoid_grad(np.dot(data,W1)+b1)) 
    gradsigmoid=np.multiply(gradh,logistic._pdf(np.dot(data,W1)+b1)) 
    gradW1=np.dot(data.T,gradsigmoid)
    gradb1=gradsigmoid
    ### Stack gradients (do not modify)
    ### Has been Modified with a transpose and axis
    grad = np.concatenate((gradW1.flatten(), gradb1.flatten(), gradW2.flatten(), gradb2.flatten()),axis=1).T
    cost=-np.dot(labels,np.log(y_pred).T)[0,0]
    return cost, grad.T

In [122]:
N = 10
dimensions = [10, 5, 10]
data2 = np.random.randn(N, dimensions[0])   # each row will be a datum

labels2 = np.zeros((N, dimensions[2]))
for i in xrange(N):
    labels2[i,random.randint(0,dimensions[2]-1)] = 1
for j in range(N):
    data=np.matrix(data2[0])    
    labels=np.matrix(labels2[0])
    params = np.random.randn((dimensions[0] + 1) * dimensions[1] + (
    dimensions[1] + 1) * dimensions[2], )
    print params.shape
    params
    for count in xrange(3):
        cost, grad=forward_backprop(data, labels, params, dimensions)
        my_gradcheck_new(forward_backprop,data, labels, params, dimensions)    
        print cost
        grad=grad.flatten()
        for j in xrange(len(params)):
            params[j]=params[j]-(alpha*grad[0,j])

(115L,)
5.43450655877
4.77565556959
4.20127705162
(115L,)
1.66281492482
1.14962211471
0.81196500196
(115L,)
2.48534465306
1.56707659553
0.953691182306
(115L,)
2.95393387306
2.38649972918
1.9064523641
(115L,)
3.47824084049
2.68877543459
2.01375090017
(115L,)
4.79790724906
3.49963860429
2.48263234369
(115L,)
3.25609074354
2.78474915346
2.47191229228
(115L,)
3.3037284478
2.61306290471
2.21288326252
(115L,)
3.47329601212
2.41153637518
1.53539229708
(115L,)
2.26403613219
1.65373572828
1.20011851691


In [101]:
def my_gradcheck_new(lossfunction,data, labels, params, dimensions):
    count=0
    it = np.nditer(params, flags=['multi_index'], op_flags=['readwrite'])
    eps=1e-5 
    while not it.finished:
        ix = it.multi_index
        params2=params
        params2[ix]=params[ix]+eps
        cost1=lossfunction(data, labels, params2, dimensions)
        params2[ix]=params[ix]-eps
        cost2=lossfunction(data, labels, params2, dimensions)
        numgrad=(cost1[0]-cost2[0])/(eps)
        reldiff = abs(numgrad - grad[0,ix]) / max(1, abs(numgrad), abs(grad[0,ix]))
        #print reldiff
        if reldiff > 1e-4:
            count+=1
            print "First gradient error found at index %s" % str(ix)
            print "Relative Difference is %f" %reldiff
            print "Your gradient: %f \t Numerical gradient: %f" % (grad[0,ix], numgrad)
        it.iternext() # Step to next dimension
    

In [64]:
my_gradcheck_new(forward_backprop,data, labels, params, dimensions)

count 0


In [109]:
print data[0]

[-0.37927885 -0.51994342  1.20641448 -1.21785846  0.44703048  1.30024322
 -0.32617239  0.97650261 -0.04435326 -0.09955904]
